In [2]:
#import

import duckdb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 100)

# query

In [3]:
con = duckdb.connect("./data/diamonds_train.db",read_only=True)
con

In [4]:
con = duckdb.connect(database="./data/diamonds_train.db", read_only=True)
query = "SHOW TABLES"
tables = con.execute(query).fetchall()

for table in tables:
    print(table)


('diamonds_city',)
('diamonds_clarity',)
('diamonds_color',)
('diamonds_cut',)
('diamonds_dimensions',)
('diamonds_properties',)
('diamonds_transactional',)


In [5]:
import duckdb
import pandas as pd

con = duckdb.connect(database="./data/diamonds_train.db", read_only=True)

# Realizar uniones utilizando SQL en DuckDB
query = """
SELECT dt.index_id, dt.price, dt.carat, dd.depth, dd.table, dd.x, dd.y, dd.z, 
       dc.city, dp.cut_id, dp.color_id, dp.clarity_id,
       dcl.clarity, dco.color, dcut.cut
FROM diamonds_transactional dt
INNER JOIN diamonds_dimensions dd ON dt.index_id = dd.index_id
INNER JOIN diamonds_city dc ON dt.city_id = dc.city_id
INNER JOIN diamonds_properties dp ON dt.index_id = dp.index_id
INNER JOIN diamonds_clarity dcl ON dp.clarity_id = dcl.clarity_id
INNER JOIN diamonds_color dco ON dp.color_id = dco.color_id
INNER JOIN diamonds_cut dcut ON dp.cut_id = dcut.cut_id

"""

df = con.execute(query).df()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index_id    40455 non-null  object 
 1   price       40455 non-null  int64  
 2   carat       40455 non-null  float64
 3   depth       40455 non-null  float64
 4   table       40455 non-null  float64
 5   x           40455 non-null  float64
 6   y           40455 non-null  float64
 7   z           40455 non-null  float64
 8   city        40455 non-null  object 
 9   cut_id      40455 non-null  object 
 10  color_id    40455 non-null  object 
 11  clarity_id  40455 non-null  object 
 12  clarity     40455 non-null  object 
 13  color       40455 non-null  object 
 14  cut         40455 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 4.6+ MB


# check df

In [6]:
# COMPROBACION DE NULOS

for columna in df.columns:
    nulos = df[columna].isnull().sum()
    unicos = df[columna].nunique()
    print(f"Columna {columna}: {nulos} nulos, {unicos} valores únicos")


Columna index_id: 0 nulos, 40455 valores únicos
Columna price: 0 nulos, 10378 valores únicos
Columna carat: 0 nulos, 270 valores únicos
Columna depth: 0 nulos, 175 valores únicos
Columna table: 0 nulos, 121 valores únicos
Columna x: 0 nulos, 543 valores únicos
Columna y: 0 nulos, 539 valores únicos
Columna z: 0 nulos, 363 valores únicos
Columna city: 0 nulos, 13 valores únicos
Columna cut_id: 0 nulos, 5 valores únicos
Columna color_id: 0 nulos, 7 valores únicos
Columna clarity_id: 0 nulos, 8 valores únicos
Columna clarity: 0 nulos, 8 valores únicos
Columna color: 0 nulos, 7 valores únicos
Columna cut: 0 nulos, 5 valores únicos


In [7]:
df.to_csv('./data/diamonds_full.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index_id    40455 non-null  object 
 1   price       40455 non-null  int64  
 2   carat       40455 non-null  float64
 3   depth       40455 non-null  float64
 4   table       40455 non-null  float64
 5   x           40455 non-null  float64
 6   y           40455 non-null  float64
 7   z           40455 non-null  float64
 8   city        40455 non-null  object 
 9   cut_id      40455 non-null  object 
 10  color_id    40455 non-null  object 
 11  clarity_id  40455 non-null  object 
 12  clarity     40455 non-null  object 
 13  color       40455 non-null  object 
 14  cut         40455 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 4.6+ MB


# CLEANING DATAFRAME

In [9]:
columns_to_drop = [column for column in df.columns if '_id' in column]

# Eliminar las columnas de df
df_cleaned = df.drop(columns=columns_to_drop)

df_cleaned.head()

df_cleaned.to_csv('./data/df_cleaned.csv')

# GET DUMMIES

In [10]:

dummies_df = pd.get_dummies(df_cleaned, columns=['city', 'clarity', 'color', 'cut'])

# Mostrar las primeras filas para verificar
dummies_df.head()


,price,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False
1,505,0.32,63.0,57.0,4.35,4.38,2.75,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False
3,738,0.41,63.8,56.0,4.68,4.72,3.00,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False


In [11]:
dummies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               40455 non-null  int64  
 1   carat               40455 non-null  float64
 2   depth               40455 non-null  float64
 3   table               40455 non-null  float64
 4   x                   40455 non-null  float64
 5   y                   40455 non-null  float64
 6   z                   40455 non-null  float64
 7   city_Amsterdam      40455 non-null  bool   
 8   city_Antwerp        40455 non-null  bool   
 9   city_Dubai          40455 non-null  bool   
 10  city_Kimberly       40455 non-null  bool   
 11  city_Las Vegas      40455 non-null  bool   
 12  city_London         40455 non-null  bool   
 13  city_Luxembourg     40455 non-null  bool   
 14  city_Madrid         40455 non-null  bool   
 15  city_New York City  40455 non-null  bool   
 16  city

In [12]:
columns_to_drop = ['city_Amsterdam', 'clarity_I1','color_D','cut_Fair']
dummies_df = dummies_df.drop(columns=columns_to_drop)
dummies_df.to_csv('./data/diamonds_dummies.csv')

In [13]:
# FEATURES + TARGET

X = dummies_df.drop('price' ,axis=1)
y = dummies_df['price']
print(X.shape,y.shape)

(40455, 35) (40455,)
